### Cargamos las librerías

In [ ]:
# Cargando las librerías
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Exploración inicial de los datos

In [ ]:
# Función para cargar y mostrar información básica del dataset
def cargar_datos(path):
    df = pd.read_csv(path)
    print(df.columns)
    print(df.shape)
    df.info()
    return df

### Creamos copia del DF

In [ ]:
# Función para crear una copia del DataFrame
def crear_copia(df):
    return df.copy()

### Imputación de valores faltantes en las columnas

In [ ]:
# Función para mostrar y filtrar valores faltantes
def mostrar_valores_faltantes(df):
    missing_values = df.isnull().sum()
    for col, value in missing_values.items():
        if value > 0:
            print(f'Variable: {col}, Datos faltantes: {value}')

# Función para imputar valores (moda para categóricas y mediana para numéricas)
def imputar_valores(df, columnas_moda, columnas_mediana):
    # Imputar con la moda
    for col in columnas_moda:
        df[col] = df[col].fillna(df[col].mode()[0])
    # Imputar con la mediana
    for col in columnas_mediana:
        df[col] = df[col].fillna(df[col].median())
    return df

### Detección de valores erróneos

In [ ]:
# Función para detectar valores erróneos
def detect_errors(df, column, rule):
    return df[~df[column].apply(rule)]  # Usamos ~ para invertir la lógica

### Eliminación de valores erróneos e imputación de la columna específica

In [ ]:
# Función para eliminar valores erróneos e imputar 'time_to_aGvHD_III_IV'
def eliminar_e_imputar(df, error_rules):
    bmtch_df_clean = df.copy()
    indices_a_eliminar = []
    
    for col, rule in error_rules.items():
        if col != 'time_to_aGvHD_III_IV':
            errores = detect_errors(bmtch_df_clean, col, rule)
            if len(errores) > 0:
                print(f"Variable: {col}, Valores erróneos detectados: {len(errores)}")
                indices_a_eliminar.extend(errores.index.tolist())
    
    bmtch_df_clean.drop(index=set(indices_a_eliminar), inplace=True)

    mediana_aGvHD = bmtch_df_clean.loc[bmtch_df_clean['time_to_aGvHD_III_IV'] != 1000000, 'time_to_aGvHD_III_IV'].median()
    bmtch_df_clean['time_to_aGvHD_III_IV'] = bmtch_df_clean['time_to_aGvHD_III_IV'].replace(1000000, mediana_aGvHD)

    return bmtch_df_clean

### Graficar histogramas de las variables numéricas seleccionadas

In [ ]:
# Función para graficar histogramas
def graficar_histogramas(df, columnas, title_prefix='Histograma de'):
    plt.figure(figsize=(12, 10))
    for i, col in enumerate(columnas, 1):
        plt.subplot(5, 2, i)
        plt.hist(df[col], bins=30, color='blue', edgecolor='black')
        plt.title(f'{title_prefix} {col}')
        plt.xlabel(col)
        plt.ylabel('Frecuencia')
    plt.tight_layout()
    plt.show()

### Aplicar transformaciones

In [ ]:
# Función para aplicar transformaciones logarítmicas y de raíz cuadrada
def aplicar_transformaciones(df, log_columns, sqrt_columns):
    for col in log_columns:
        df[col] = np.log1p(df[col])
    for col in sqrt_columns:
        df[f'sqrt_{col}'] = np.sqrt(df[col])
    return df

### Graficar Boxplots de las variables numéricas seleccionadas

In [ ]:
# Función para graficar boxplots
def graficar_boxplots(df, columnas):
    n_cols = 5
    n_rows = (len(columnas) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
    for i, col in enumerate(columnas):
        row = i // n_cols
        col_num = i % n_cols
        sns.boxplot(data=df, y=col, ax=axes[row, col_num], color="skyblue")
        axes[row, col_num].set_title(f'Boxplot de {col}')
    for j in range(i + 1, n_rows * n_cols):
        fig.delaxes(axes.flatten()[j])
    plt.tight_layout()
    plt.show()

### Reemplazamos valores faltantes

In [ ]:
# Función para reemplazar outliers con NaN utilizando el IQR
def replace_outliers_with_nan(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].apply(lambda x: x if lower_bound <= x <= upper_bound else np.nan)

### Graficar Barras

In [ ]:
# Función para graficar barras de variables categóricas
def graficar_barras(df, columnas):
    n_cols = 5
    n_rows = (len(columnas) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 15))
    fig.tight_layout(pad=5.0)
    for i, col in enumerate(columnas):
        row = i // n_cols
        col_num = i % n_cols
        values = df[col].value_counts()
        axes[row, col_num].bar(values.index, values.values, color="blue")
        axes[row, col_num].set_title(f'Frecuencia de {col}')
        axes[row, col_num].tick_params(axis='x', rotation=45)
    for j in range(i + 1, n_rows * n_cols):
        fig.delaxes(axes.flatten()[j])
        plt.show()

### Matríz de correlación

In [ ]:
# Función para calcular y mostrar la matriz de correlación
def mostrar_correlacion(df, columnas, target_col):
    correlation_matrix = df[columnas].corr()
    target_corr = correlation_matrix[[target_col]]
    print(target_corr)

### Llamado de funciones

In [ ]:
# Ejemplo de uso
path = '../data/raw/bone-marrow.csv'
bmtch_df = cargar_datos(path)
bmtch_df_copy = crear_copia(bmtch_df)
mostrar_valores_faltantes(bmtch_df_copy)

# Listas de columnas a imputar
columnas_moda = ['RecipientABO', 'RecipientRh', 'ABOmatch', 'CMVstatus', 'DonorCMV', 'RecipientCMV', 'Antigen', 'Allele', 'extcGvHD']
columnas_mediana = ['CD3dCD34', 'CD3dkgx10d8', 'Rbodymass']
bmtch_df_copy = imputar_valores(bmtch_df_copy, columnas_moda, columnas_mediana)

# Diccionario de reglas de error
error_rules = {
    'Recipientgender': lambda x: x in [0, 1],  # 0: Femenino, 1: Masculino
    'Stemcellsource': lambda x: x in [0, 1],  # 0: Sangre periférica, 1: Médula ósea
    'Donorage': lambda x:  (0 <= x <= 120),  # Edad del donante fuera del rango
    'Donorage35': lambda x: x in [0, 1],  # Edad del donante fuera del rango
    'IIIV': lambda x: x in [0, 1],  # Clasificación del trasplante entre I y IV
    'Gendermatch': lambda x: x in [0, 1],  # 0: No compatible, 1: Compatible
    'DonorABO': lambda x: x in [-1, 0, 1, 2],  # -1: B, 0: O, 1: A, 2: AB
    'RecipientABO': lambda x: x in [-1, 0, 1, 2],  # -1: B, 0: O, 1: A, 2: AB
    'RecipientRh': lambda x: x in [0, 1],  # 0: Rh-, 1: Rh+
    'ABOmatch': lambda x: x in [0, 1],  # 0: Mismatched, 1: Matched
    'CMVstatus': lambda x: x in [0, 1, 2, 3],  # Presencia/ausencia de CMV y otros estados
    'DonorCMV': lambda x: x in [0, 1],  # 0: Negativo, 1: Positivo
    'RecipientCMV': lambda x: x in [0, 1],  # 0: Negativo, 1: Positivo
    'Disease': lambda x: x in ['ALL', 'AML', 'chronic', 'nonmalignant', 'lymphoma'],  # Tipos de enfermedad
    'Riskgroup': lambda x: x in [0, 1],  # 0: Bajo, 1: Alto
    'Txpostrelapse': lambda x: x in [0, 1],  # 0: No, 1: Sí (trasplante tras recaída)
    'Diseasegroup': lambda x: x in [0, 1],  # 0: No maligna, 1: Maligna
    'HLAmatch': lambda x: x in [0, 1, 2, 3],  # Compatibilidad HLA 7/10 a 10/10
    'HLAmismatch': lambda x: x in [0, 1],  # 0: Compatible, 1: No compatible
    'Antigen': lambda x: x  in [-1, 0, 1, 2],  # Diferencia de antígenos entre donante y receptor
    'Allele': lambda x: x  in [-1, 0, 1, 2, 3],  # Diferencia de alelos entre donante y receptor
    'HLAgrI': lambda x:  (0 <= x <= 5),  # Grado de diferencia HLA (fuera de rango)
    'Recipientage': lambda x:  (0 <= x <= 120),  # Edad del receptor fuera de rango
    'Recipientage10': lambda x: x in [0, 1],  # 0: >= 10 años, 1: < 10 años
    'Recipientageint': lambda x: x in [0, 1, 2],  # Edad del receptor discretizada en intervalos
    'Relapse': lambda x: x in [0, 1],  # 0: No, 1: Sí (recaída)
    'aGvHDIIIIV': lambda x: x in [0, 1, 2, 3],  # Grado de GvHD aguda entre 0 y IV
    'extcGvHD': lambda x: x in [0, 1],  # 0: No, 1: Sí (GvHD crónica extensa)
    'CD34kgx10d6': lambda x:  (0 <= x <= 60),  # Dosis de CD34+ fuera de rango
    'CD3dCD34': lambda x:  (0 <= x <= 100),  # Relación CD3/CD34 fuera de rango
    'CD3dkgx10d8': lambda x:  (0 <= x <= 21),  # Dosis de CD3+ fuera de rango
    'Rbodymass': lambda x:  (6 <= x <= 104),  # Masa corporal del receptor fuera de rango
    'ANCrecovery': lambda x:  (7 <= x <= 30),  # Tiempo de recuperación de neutrófilos fuera de rango
    'PLTrecovery': lambda x:  (0 <= x <= 365),  # Tiempo de recuperación de plaquetas fuera de rango
    'time_to_aGvHD_III_IV': lambda x:  (0 <= x <= 365),  # Tiempo en días hasta GvHD aguda
    'survival_time': lambda x:  (0 <= x <= 4000),  # Tiempo de supervivencia fuera de rango
    'survival_status': lambda x: x in [0, 1],  # 0: Vivo, 1: Fallecido
}

# Eliminar valores erróneos e imputar 'time_to_aGvHD_III_IV'
bmtch_df_clean = eliminar_e_imputar(bmtch_df_copy, error_rules)

# Graficar histogramas
selected_columns = ['Donorage', 'Recipientage', 'Rbodymass', 'CD34kgx10d6', 'CD3dkgx10d8', 'CD3dCD34', 'ANCrecovery', 'PLTrecovery', 'time_to_aGvHD_III_IV', 'survival_time']
graficar_histogramas(bmtch_df_clean, selected_columns)

# Aplicar transformaciones
log_columns = ['Rbodymass', 'CD34kgx10d6', 'CD3dkgx10d8', 'CD3dCD34', 'ANCrecovery', 'PLTrecovery']
sqrt_columns = ['time_to_aGvHD_III_IV', 'survival_time']
bmtch_df_clean = aplicar_transformaciones(bmtch_df_clean, log_columns, sqrt_columns)

# Graficar boxplots
graficar_boxplots(bmtch_df_clean, selected_columns)

# Reemplazar outliers
columns_to_clean = ['Rbodymass', 'CD34kgx10d6', 'CD3dkgx10d8', 'CD3dCD34', 'ANCrecovery', 'PLTrecovery', 'time_to_aGvHD_III_IV', 'survival_time']
for col in columns_to_clean:
    replace_outliers_with_nan(bmtch_df_clean, col)

# Graficar barras para variables categóricas
categorical_columns = ['Recipientgender', 'Stemcellsource', 'Donorage35', 'IIIV', 'Gendermatch']
graficar_barras(bmtch_df_clean, categorical_columns)

# Mostrar correlación
correlacion_columnas = ['Donorage', 'Recipientage', 'Rbodymass', 'CD34kgx10d6', 'CD3dkgx10d8', 'CD3dCD34', 'ANCrecovery', 'PLTrecovery', 'time_to_aGvHD_III_IV', 'survival_time']
mostrar_correlacion(bmtch_df_clean, correlacion_columnas, 'survival_time')